# Fraud Detection Project – Data Cleaning Notebook

This notebook focuses **only** on cleaning and preparing the raw data.
We will handle:
- Missing values
- Duplicates
- Inconsistent text (e.g., CHANNEL: "Mobile", "mobile", "MOBILE")
- Invalid/future timestamps
- Boolean & numeric coercion
- Recompute `location_mismatch`
- Final clean dataset saved for EDA/modeling

In [117]:
import os
os.getcwd()

'C:\\Users\\HP ELITEBOOK 830 G5\\Desktop\\cybox-pay\\notebooks'

In [118]:
import os
os.listdir("../data")


['cybox_pay_fraud_boost .csv', 'cybox_pay_transcations.csv', 'merged_data.csv']

In [119]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Loading datasets...")
data1 = pd.read_csv("../data/cybox_pay_transcations.csv")
data2 = pd.read_csv("../data/cybox_pay_fraud_boost .csv")

print(f"Data1 shape: {data1.shape}")
print(f"Data2 shape: {data2.shape}")

Loading datasets...
Data1 shape: (10200, 26)
Data2 shape: (1200, 26)


### Step 1: Define Final Schema & Merge Safely

In [120]:
def check_columns(df, name): # Only reindex after confirming missing columns are expected.
    missing = set(final_columns) - set(df.columns)
    extra = set(df.columns) - set(final_columns)

    print(f"\n{name}")
    print(f"Missing columns: {missing}")
    print(f"Extra columns: {extra}")

check_columns(data1, "data1")
check_columns(data2, "data2")



data1
Missing columns: set()
Extra columns: set()

data2
Missing columns: set()
Extra columns: set()


In [121]:
final_columns = [
    'transaction_id', 'customer_id', 'timestamp', 'home_country', 'source_currency',
    'dest_currency', 'channel', 'amount_src', 'amount_usd', 'fee', 'exchange_rate_src_to_dest',
    'device_id', 'new_device', 'ip_address', 'ip_country', 'location_mismatch',
    'ip_risk_score', 'kyc_tier', 'account_age_days', 'device_trust_score',
    'chargeback_history_count', 'risk_score_internal', 'txn_velocity_1h',
    'txn_velocity_24h', 'corridor_risk', 'is_fraud'
]

# Reindex both datasets to same columns
data1 = data1.reindex(columns=final_columns)
data2 = data2.reindex(columns=final_columns)

# Concatenate
df = pd.concat([data1, data2], ignore_index=True)
print(f"Merged shape: {df.shape}")


Merged shape: (11400, 26)


### Step 2: Initial Inspection

In [122]:
print("=== Missing Values Before Cleaning ===")
print(df.isnull().sum())

print("\n=== Data Types ===")
print(df.dtypes)

print("\n=== Sample Data ===")
display(df.head(3))

=== Missing Values Before Cleaning ===
transaction_id                 0
customer_id                    0
timestamp                     29
home_country                   0
source_currency                0
dest_currency                  0
channel                        0
amount_src                     0
amount_usd                   305
fee                          295
exchange_rate_src_to_dest      0
device_id                      0
new_device                     0
ip_address                   305
ip_country                   301
location_mismatch              0
ip_risk_score                  0
kyc_tier                     300
account_age_days               0
device_trust_score           295
chargeback_history_count       0
risk_score_internal            0
txn_velocity_1h                0
txn_velocity_24h               0
corridor_risk                  0
is_fraud                       0
dtype: int64

=== Data Types ===
transaction_id                object
customer_id                   obj

,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,exchange_rate_src_to_dest,device_id,new_device,ip_address,ip_country,location_mismatch,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
0,fee8542d-8ee6-4b0d-9671-c294dd08ed26,402cccc9-28de-45b3-9af7-cc5302aa1f93,2022-10-03 18:40:59.468549+00:00,US,USD,CAD,ATM,278.19,278.19,4.25,1.351351,9f292dcc-3297-4947-a260-6a1ef69041ff,False,221.78.171.180,US,False,0.123,standard,263,0.522,0,0.223,0,0,0.0,0
1,bfdb9fc1-27fe-4a85-b043-4d813d679259,67c2c6b3-ef0a-4777-a3f1-c84a851bb6ad,2022-10-03 20:39:38.468549+00:00,CA,CAD,MXN,web,208.51,154.29,4.24,12.758621,3a95b9f5-309f-4684-a46d-e2ff2435bf78,True,120.12.20.29,CA,False,0.569,standard,947,0.475,0,0.268,0,1,0.0,0
2,fc855034-3ea5-4993-9afa-b511d93fe5e8,6d0d9b27-fa26-45f8-93b1-2df29d182d9c,2022-10-03 23:02:43.468549+00:00,US,USD,CNY,mobile,160.33,160.33,2.70,7.142857,a4737752-9aac-43ed-9d8b-2ccdffc24052,False,223.96.181.93,US,False,0.437,enhanced,367,0.939,0,0.176,0,0,0.0,0


### Step 3: Clean `transaction_id` & Remove Duplicates
We use `transaction_id` as unique identifier → remove duplicates

In [123]:
print(f"Total rows before deduplication: {len(df)}")
df = df.drop_duplicates(subset=['transaction_id'], keep='first')
print(f"After removing duplicates: {len(df)}")
print(f"Removed {len(data1) + len(data2) - len(df)} duplicate transactions")



#drop_duplicates(...) removes repeated rows 
# based on a rule defined.

#subset=['transaction_id'] means: “treat rows as duplicates if they share the same transaction_id.”

# keep='first' means: if there are duplicates, keep the first one you see and drop the others.

Total rows before deduplication: 11400
After removing duplicates: 11200
Removed 200 duplicate transactions


In [124]:
df.head()

,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,exchange_rate_src_to_dest,device_id,new_device,ip_address,ip_country,location_mismatch,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud
0,fee8542d-8ee6-4b0d-9671-c294dd08ed26,402cccc9-28de-45b3-9af7-cc5302aa1f93,2022-10-03 18:40:59.468549+00:00,US,USD,CAD,ATM,278.19,278.19,4.25,1.351351,9f292dcc-3297-4947-a260-6a1ef69041ff,False,221.78.171.180,US,False,0.123,standard,263,0.522,0,0.223,0,0,0.0,0
1,bfdb9fc1-27fe-4a85-b043-4d813d679259,67c2c6b3-ef0a-4777-a3f1-c84a851bb6ad,2022-10-03 20:39:38.468549+00:00,CA,CAD,MXN,web,208.51,154.29,4.24,12.758621,3a95b9f5-309f-4684-a46d-e2ff2435bf78,True,120.12.20.29,CA,False,0.569,standard,947,0.475,0,0.268,0,1,0.0,0
2,fc855034-3ea5-4993-9afa-b511d93fe5e8,6d0d9b27-fa26-45f8-93b1-2df29d182d9c,2022-10-03 23:02:43.468549+00:00,US,USD,CNY,mobile,160.33,160.33,2.70,7.142857,a4737752-9aac-43ed-9d8b-2ccdffc24052,False,223.96.181.93,US,False,0.437,enhanced,367,0.939,0,0.176,0,0,0.0,0
3,2cf8c08e-42ec-444d-a755-34b9a2a0a4ca,7bd5200c-5d19-44f0-9afe-8b339a05366b,2022-10-04 01:08:53.468549+00:00,US,USD,EUR,mobile,59.41,59.41,2.22,0.925926,6aeb85a3-5603-4221-896c-9e6882764f1a,False,186.228.15.74,US,False,0.594,standard,147,0.551,0,0.391,0,0,0.0,0
4,d907a74d-b426-438d-97eb-dbe911aca91c,70a93d26-8e3a-4179-900c-a4a7a74d08e5,2022-10-04 09:35:03.468549+00:00,US,USD,INR,mobile,200.96,200.96,3.61,83.333333,a5b9250e-dbe0-4c5f-a6e7-5492b7349402,False,11.82.47.62,US,False,0.121,enhanced,257,0.894,0,0.257,0,0,0.0,0


### Step 4: Fix Timestamp (UTC + Drop Future/Invalid)
Convert to datetime with UTC and coerce errors

In [125]:

# 1) Parse timestamps safely and consistently (UTC)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True, errors='coerce')

# 2) Report invalid timestamps created by coercion
invalid_ts = df['timestamp'].isna().sum()
print(f"Invalid timestamps (NaT): {invalid_ts}")

# 3) Define a sensible cutoff date (use 'now' in UTC so it always stays correct)
current_date = pd.Timestamp.now(tz='UTC')
future_mask = df['timestamp'] > current_date

print(f"Current UTC cutoff: {current_date}")
print(f"Future timestamps found: {future_mask.sum()} → dropping")

# 4) Drop future timestamps
df = df.loc[~future_mask].copy()

# 5) Drop rows with missing timestamps (NaT)
before_drop = len(df)
df = df.dropna(subset=['timestamp']).copy()
after_drop = len(df)

print(f"Dropped {before_drop - after_drop} rows with invalid timestamps (NaT)")
print(f"Shape after timestamp cleaning: {df.shape}")



Invalid timestamps (NaT): 60
Current UTC cutoff: 2025-12-16 02:32:51.391802+00:00
Future timestamps found: 0 → dropping
Dropped 60 rows with invalid timestamps (NaT)
Shape after timestamp cleaning: (11140, 26)


### Step 5: Extract Time Features

In [126]:

# Extract the hour of the day (0–23) from the transaction timestamp
df['hour'] = df['timestamp'].dt.hour

# Extract the day of the week from the timestamp (Monday=0, Sunday=6)
df['day_of_week'] = df['timestamp'].dt.dayofweek

# Create a binary flag for weekend transactions (1 = Saturday/Sunday, 0 = weekday)
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Flag transactions that occur late at night or early morning (00:00–05:59)
# These hours are commonly associated with higher fraud risk
df['is_night'] = ((df['hour'] >= 0) & (df['hour'] <= 5)).astype(int)

# Extract the month of the transaction (1–12) to capture seasonal patterns
df['month'] = df['timestamp'].dt.month

# Extract the day of the month (1–31) to capture payday or end-of-month behaviour
df['day'] = df['timestamp'].dt.day

# Log the newly created time-based features for tracking and debugging
print(
    "Time features created:",
    ['hour', 'day_of_week', 'is_weekend', 'is_night', 'month', 'day']
)


Time features created: ['hour', 'day_of_week', 'is_weekend', 'is_night', 'month', 'day']


### Step 6: Standardize Categorical Columns (Fix Inconsistencies)


In [127]:
# Inspect the raw 'channel' values to identify casing, spacing, or formatting issues
print("Before CHANNEL cleaning:")
print(df['channel'].unique()[:20])

# Convert channel values to strings, remove leading/trailing spaces, and standardise to uppercase
df['channel'] = (
    df['channel']
    .astype(str)      # ensure all values are strings to avoid errors
    .str.strip()      # remove extra spaces
    .str.upper()      # standardise casing
)

# Verify that the cleaning worked as expected
print("After CHANNEL cleaning:")
print(df['channel'].unique())

# List of other categorical/text columns that require the same standardisation
text_cols = ['home_country', 'source_currency', 'dest_currency', 'ip_country', 'kyc_tier']

# Loop through each categorical column and clean it consistently
for col in text_cols:
    if col in df.columns:  # safety check in case a column is missing
        df[col] = (
            df[col]
            .astype(str)      # convert values to strings
            .str.strip()      # remove extra spaces
            .str.upper()      # standardise casing
        )
        
        # Replace string representations of missing values with proper NaN
        df[col] = df[col].replace({
            'NAN': np.nan,
            '<NA>': np.nan
        })

# Confirm that all categorical columns have been cleaned and standardised
print("All categorical columns standardised to UPPERCASE and cleaned")


Before CHANNEL cleaning:
['ATM' 'web' 'mobile' 'WEB' ' web  ' 'MOBILE' 'mobille' 'unknown'
 ' mobile  ' 'weeb' 'ATm' ' ATM  ']
After CHANNEL cleaning:
['ATM' 'WEB' 'MOBILE' 'MOBILLE' 'UNKNOWN' 'WEEB']
All categorical columns standardised to UPPERCASE and cleaned


##### Uppercasing won’t fix WEEB → WEB or MOBILLE → MOBILE, so we need a standardisation map (rule-based recoding).

In [128]:
#  Clean basic formatting first (you already did this, but keep it here for safety)
df['channel'] = df['channel'].astype(str).str.strip().str.upper()

#  Fix known typos / variants by mapping them to the correct canonical values
channel_map = {
    "WEEB": "WEB",
    "MOBILLE": "MOBILE",
    "ATM ": "ATM",     # just in case any trailing-space survived earlier steps
    "AT M": "ATM",     # example of weird spacing
}

df['channel'] = df['channel'].replace(channel_map)

#  Optional: collapse anything unexpected into "UNKNOWN"
allowed = {"ATM", "WEB", "MOBILE", "UNKNOWN"}
df['channel'] = df['channel'].where(df['channel'].isin(allowed), "UNKNOWN")

#  Check results
print("After typo-fix CHANNEL cleaning:")
print(df['channel'].value_counts(dropna=False))
print(df['channel'].unique())


After typo-fix CHANNEL cleaning:
channel
MOBILE     6358
WEB        3749
ATM         996
UNKNOWN      37
Name: count, dtype: int64
['ATM' 'WEB' 'MOBILE' 'UNKNOWN']


##### Before moving on, inspect how risky UNKNOWN actually is:

In [129]:
df.groupby('channel')['is_fraud'].mean().sort_values(ascending=False)  # This does not mean: “What percentage of all
# fraud comes from each channel?” It means:“Within each channel, what proportion of transactions are fraudulent?”

# So for WEB = 0.163510, read it as:16.35% of WEB transactions are fraudulent

#Each channel is calculated independently, so there’s no reason for the values to sum to 1 (100%).


channel
WEB        0.163510
ATM        0.081325
MOBILE     0.046870
UNKNOWN    0.027027
Name: is_fraud, dtype: float64

##### Other categorical columns that need fixing 

In [130]:
cols_to_inspect = ['home_country', 'ip_country', 'kyc_tier']

for col in cols_to_inspect:
    if col in df.columns:
        print(f"\n--- {col.upper()} ---")
        
        # show sample of raw unique values
        print("Sample unique values:")
        print(df[col].dropna().unique()[:20])
        
        # show counts including missing values
        print("\nValue counts (top 10, incl. NaN):")
        print(df[col].value_counts(dropna=False).head(10))



--- HOME_COUNTRY ---
Sample unique values:
['US' 'CA' 'UK' 'UNKNOWN']

Value counts (top 10, incl. NaN):
home_country
US         7810
UK         2092
CA         1206
UNKNOWN      32
Name: count, dtype: int64

--- IP_COUNTRY ---
Sample unique values:
['US' 'CA' 'UK' 'UNKNOWN']

Value counts (top 10, incl. NaN):
ip_country
US         6726
UK         2376
CA         1708
NaN         299
UNKNOWN      31
Name: count, dtype: int64

--- KYC_TIER ---
Sample unique values:
['STANDARD' 'ENHANCED' 'LOW' 'STANDRD' 'UNKNOWN' 'ENHANCD']

Value counts (top 10, incl. NaN):
kyc_tier
STANDARD    7867
ENHANCED    1818
LOW         1043
NaN          298
STANDRD       72
UNKNOWN       30
ENHANCD       12
Name: count, dtype: int64


In [131]:


# HOME_COUNTRY: keep as is, but make sure missing/blank becomes UNKNOWN
df['home_country'] = df['home_country'].astype(str).str.strip().str.upper()
df['home_country'] = df['home_country'].replace({'NAN': np.nan, '<NA>': np.nan, '': np.nan})
df['home_country'] = df['home_country'].fillna('UNKNOWN')

#  IP_COUNTRY: standardise + convert missing to UNKNOWN
df['ip_country'] = df['ip_country'].astype(str).str.strip().str.upper()
df['ip_country'] = df['ip_country'].replace({'NAN': np.nan, '<NA>': np.nan, '': np.nan})
df['ip_country'] = df['ip_country'].fillna('UNKNOWN')

#  KYC_TIER: standardise + fix spelling variants
df['kyc_tier'] = df['kyc_tier'].astype(str).str.strip().str.upper()
df['kyc_tier'] = df['kyc_tier'].replace({'NAN': np.nan, '<NA>': np.nan, '': np.nan})
df['kyc_tier'] = df['kyc_tier'].fillna('UNKNOWN')

kyc_map = {
    'STANDRD': 'STANDARD',
    'ENHANCND':'ENHANCED',
    'ENHANCD': 'ENHANCED'
}
df['kyc_tier'] = df['kyc_tier'].replace(kyc_map)

# Optional: enforce allowed tiers (anything else becomes UNKNOWN)
#allowed_tiers = {'LOW', 'STANDARD', 'ENHANCED', 'UNKNOWN'}
#df['kyc_tier'] = df['kyc_tier'].where(df['kyc_tier'].isin(allowed_tiers), 'UNKNOWN')

#  Quick verification
print("home_country:", df['home_country'].value_counts(dropna=False))
print("\nip_country:", df['ip_country'].value_counts(dropna=False))
print("\nkyc_tier:", df['kyc_tier'].value_counts(dropna=False))
print("\nKYC unique values:", df['kyc_tier'].unique())


home_country: home_country
US         7810
UK         2092
CA         1206
UNKNOWN      32
Name: count, dtype: int64

ip_country: ip_country
US         6726
UK         2376
CA         1708
UNKNOWN     330
Name: count, dtype: int64

kyc_tier: kyc_tier
STANDARD    7939
ENHANCED    1830
LOW         1043
UNKNOWN      328
Name: count, dtype: int64

KYC unique values: ['STANDARD' 'ENHANCED' 'LOW' 'UNKNOWN']


### Step 7: Fix Boolean Columns (`new_device`, `location_mismatch`)


In [132]:

# Columns that should behave as boolean flags (yes/no type signals)
bool_cols = ['new_device', 'location_mismatch']

# Loop through each boolean-like column
for col in bool_cols:
    
    # Inspect raw values before cleaning to understand inconsistencies
    print(f"\nBefore fixing {col}:")
    print(df[col].value_counts(dropna=False).head())

    # Convert values to string and standardise casing for reliable mapping
    df[col] = (
        df[col]
        .astype(str)      # ensure values are strings
        .str.strip()      # remove leading/trailing spaces
        .str.upper()      # standardise to uppercase
    )

    # Map all known boolean representations to numeric 1/0
    df[col] = df[col].map({
        'TRUE': 1, 'FALSE': 0,   # textual booleans
        'YES': 1, 'NO': 0,       # yes/no formats
        '1': 1, '0': 0,          # integer strings
        '1.0': 1, '0.0': 0,      # float strings
        'T': 1, 'F': 0           # shorthand flags
    })

    # Count how many values could not be mapped (will be treated as False)
    unmapped_count = df[col].isna().sum()
    print(f"{col}: {unmapped_count} values were unmapped and set to 0")

    # Replace unmapped or missing values with 0 (False) and enforce integer type
    df[col] = df[col].fillna(0).astype(int)

# Confirm completion of boolean standardisation
print("Boolean columns converted to clean 0/1 flags")




Before fixing new_device:
new_device
False    9810
True     1330
Name: count, dtype: int64
new_device: 0 values were unmapped and set to 0

Before fixing location_mismatch:
location_mismatch
False    9322
True     1818
Name: count, dtype: int64
location_mismatch: 0 values were unmapped and set to 0
Boolean columns converted to clean 0/1 flags


In [133]:
pd.set_option('display.max_columns', None)
df.head()


,transaction_id,customer_id,timestamp,home_country,source_currency,dest_currency,channel,amount_src,amount_usd,fee,exchange_rate_src_to_dest,device_id,new_device,ip_address,ip_country,location_mismatch,ip_risk_score,kyc_tier,account_age_days,device_trust_score,chargeback_history_count,risk_score_internal,txn_velocity_1h,txn_velocity_24h,corridor_risk,is_fraud,hour,day_of_week,is_weekend,is_night,month,day
0,fee8542d-8ee6-4b0d-9671-c294dd08ed26,402cccc9-28de-45b3-9af7-cc5302aa1f93,2022-10-03 18:40:59.468549+00:00,US,USD,CAD,ATM,278.19,278.19,4.25,1.351351,9f292dcc-3297-4947-a260-6a1ef69041ff,0,221.78.171.180,US,0,0.123,STANDARD,263,0.522,0,0.223,0,0,0.0,0,18,0,0,0,10,3
1,bfdb9fc1-27fe-4a85-b043-4d813d679259,67c2c6b3-ef0a-4777-a3f1-c84a851bb6ad,2022-10-03 20:39:38.468549+00:00,CA,CAD,MXN,WEB,208.51,154.29,4.24,12.758621,3a95b9f5-309f-4684-a46d-e2ff2435bf78,1,120.12.20.29,CA,0,0.569,STANDARD,947,0.475,0,0.268,0,1,0.0,0,20,0,0,0,10,3
2,fc855034-3ea5-4993-9afa-b511d93fe5e8,6d0d9b27-fa26-45f8-93b1-2df29d182d9c,2022-10-03 23:02:43.468549+00:00,US,USD,CNY,MOBILE,160.33,160.33,2.70,7.142857,a4737752-9aac-43ed-9d8b-2ccdffc24052,0,223.96.181.93,US,0,0.437,ENHANCED,367,0.939,0,0.176,0,0,0.0,0,23,0,0,0,10,3
3,2cf8c08e-42ec-444d-a755-34b9a2a0a4ca,7bd5200c-5d19-44f0-9afe-8b339a05366b,2022-10-04 01:08:53.468549+00:00,US,USD,EUR,MOBILE,59.41,59.41,2.22,0.925926,6aeb85a3-5603-4221-896c-9e6882764f1a,0,186.228.15.74,US,0,0.594,STANDARD,147,0.551,0,0.391,0,0,0.0,0,1,1,0,1,10,4
4,d907a74d-b426-438d-97eb-dbe911aca91c,70a93d26-8e3a-4179-900c-a4a7a74d08e5,2022-10-04 09:35:03.468549+00:00,US,USD,INR,MOBILE,200.96,200.96,3.61,83.333333,a5b9250e-dbe0-4c5f-a6e7-5492b7349402,0,11.82.47.62,US,0,0.121,ENHANCED,257,0.894,0,0.257,0,0,0.0,0,9,1,0,0,10,4


In [134]:
for col in ['new_device', 'location_mismatch']:
    print(f"\n{col.upper()} value counts:")
    print(df[col].value_counts(dropna=False))



NEW_DEVICE value counts:
new_device
0    9810
1    1330
Name: count, dtype: int64

LOCATION_MISMATCH value counts:
location_mismatch
0    9322
1    1818
Name: count, dtype: int64


### Step 8: Recompute `location_mismatch`

In [135]:

# Recompute location_mismatch using cleaned country fields
# This avoids relying on potentially incorrect upstream flags

df['location_mismatch'] = (
    # Flag a mismatch only when home_country and ip_country are different
    (df['home_country'] != df['ip_country']) &
    
    # Exclude cases where home_country is UNKNOWN
    # Missing home country should not automatically imply a mismatch
    (df['home_country'] != 'UNKNOWN') &
    
    # Exclude cases where ip_country is UNKNOWN
    # Unknown IP location is treated as missing information, not a mismatch
    (df['ip_country'] != 'UNKNOWN')
).astype(int)  # Convert True/False to numeric 1/0 for modelling

# Log that the location mismatch feature has been rebuilt correctly
print("location_mismatch recomputed from cleaned country fields (UNKNOWN handled explicitly)")

# Display the distribution of matches vs mismatches for sanity checking
print(df['location_mismatch'].value_counts())


location_mismatch recomputed from cleaned country fields (UNKNOWN handled explicitly)
location_mismatch
0    9380
1    1760
Name: count, dtype: int64


### Step 9: Clean & Coerce Numeric Columns

In [136]:
# List of columns that are expected to be numeric fraud signals
numeric_cols = [
    'amount_src', 'amount_usd', 'fee', 'exchange_rate_src_to_dest',
    'ip_risk_score', 'account_age_days', 'device_trust_score',
    'chargeback_history_count', 'risk_score_internal',
    'txn_velocity_1h', 'txn_velocity_24h', 'corridor_risk'
]

# Loop through each numeric column and clean it consistently
for col in numeric_cols:
    if col in df.columns:  # safety check to avoid KeyErrors

        # Convert values to string so text cleaning can be applied safely
        df[col] = df[col].astype(str)

        # Remove common non-numeric characters (commas, currency symbols, percentages)
        # This handles values like "$1,200", "3,500", or "45%"
        df[col] = df[col].str.replace(r'[,$%]', '', regex=True).str.strip()

        # Convert the cleaned values to numeric type
        # Invalid values are coerced to NaN instead of causing errors
        df[col] = pd.to_numeric(df[col], errors='coerce')

        # Log how many values became NaN after coercion
        # This helps identify problematic columns early
        nan_count = df[col].isna().sum()
        print(f"{col}: {nan_count} NaN values after numeric coercion")

# Confirm completion of numeric cleaning
print("Numeric columns cleaned and coerced to numeric types")



amount_src: 0 NaN values after numeric coercion
amount_usd: 300 NaN values after numeric coercion
fee: 290 NaN values after numeric coercion
exchange_rate_src_to_dest: 0 NaN values after numeric coercion
ip_risk_score: 0 NaN values after numeric coercion
account_age_days: 0 NaN values after numeric coercion
device_trust_score: 290 NaN values after numeric coercion
chargeback_history_count: 0 NaN values after numeric coercion
risk_score_internal: 0 NaN values after numeric coercion
txn_velocity_1h: 0 NaN values after numeric coercion
txn_velocity_24h: 0 NaN values after numeric coercion
corridor_risk: 0 NaN values after numeric coercion
Numeric columns cleaned and coerced to numeric types


In [137]:
(df[['amount_usd', 'fee', 'device_trust_score']].isna().mean() * 100).round(2)


amount_usd            2.69
fee                   2.60
device_trust_score    2.60
dtype: float64

### Step 10: Handle Negative/Invalid Values


In [138]:
# These should not be negative
non_negative = ['amount_src', 'amount_usd', 'fee', 'account_age_days',
                'chargeback_history_count', 'txn_velocity_1h', 'txn_velocity_24h']

for col in non_negative:
    if col in df.columns:
        negatives = (df[col] < 0).sum()
        if negatives > 0:
            print(f"{col}: {negatives} negative values → set to 0")
            df[col] = df[col].clip(lower=0)

# Clip risk scores to [0,1] if they are probabilities
score_cols = ['ip_risk_score', 'device_trust_score', 'risk_score_internal', 'corridor_risk']
for col in score_cols:
    if col in df.columns:
        df[col] = df[col].clip(0, 1)

print("Invalid negatives and out-of-range scores fixed")

amount_src: 100 negative values → set to 0
fee: 100 negative values → set to 0
txn_velocity_1h: 200 negative values → set to 0
Invalid negatives and out-of-range scores fixed


### Step 11: Final Missing Value Imputation (Minimal & Safe)

In [139]:

print("\n Missing Values Before Final Sweep")
missing_before = df.isnull().sum()
print(missing_before[missing_before > 0])

# 1. Target column – MUST NOT be missing
if df['is_fraud'].isnull().any():
    n_missing_target = df['is_fraud'].isnull().sum()
    print(f"Warning: {n_missing_target} rows have missing is_fraud -> dropping them (critical for modeling)")
    df = df.dropna(subset=['is_fraud']).copy()

# 2. Any remaining categorical columns → fill with 'MISSING'
cat_columns = ['home_country', 'source_currency', 'dest_currency',
               'channel', 'ip_country', 'kyc_tier',
               'transaction_id', 'customer_id', 'device_id', 'ip_address']

for col in cat_columns:
    if col in df.columns:
        df[col] = df[col].fillna('MISSING').astype(str)

# 3. remaining numeric columns → median (safe & fast)
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

for col in numeric_columns:
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"Imputed {col:30s} with median → {median_val:.6f}")

# 4. Final sanity check
print("\nMissing Values AFTER Final Sweep ")
missing_after = df.isnull().sum().sum()
print(f"Total missing values = {missing_after}")

assert missing_after == 0, f"ERROR: Still {missing_after} missing values! Check the print above."

print("All missing values successfully eliminated!")




 Missing Values Before Final Sweep
amount_usd            300
fee                   290
ip_address            300
device_trust_score    290
dtype: int64
Imputed amount_usd                     with median → 163.645000
Imputed fee                            with median → 3.500000
Imputed device_trust_score             with median → 0.654500

Missing Values AFTER Final Sweep 
Total missing values = 0
All missing values successfully eliminated!


### Step 12: Final Checks & Save Clean Data

In [145]:
from pathlib import Path

assert df.duplicated(subset=['transaction_id']).sum() == 0, "Duplicates remain!"

print(f"\nFinal clean dataset")
print(f"Shape         : {df.shape}")
print(f"Fraud rate    : {df['is_fraud'].mean():.4%}")
print(f"Date range    : {df['timestamp'].min()} → {df['timestamp'].max()}")

# Save – ready for EDA and modeling

# Path to the project-level data folder
output_dir = Path("../data")

# Create it if it doesn’t exist
output_dir.mkdir(parents=True, exist_ok=True)

# Save the cleaned dataset
output_path = output_dir / "cleaned_transactions_for_eda.csv"
df.to_csv(output_path, index=False)

#display(df.head())
print(f"\nClean dataset saved to → {output_path.resolve()}")





Final clean dataset
Shape         : (11140, 32)
Fraud rate    : 8.9138%
Date range    : 2022-10-03 18:40:59.468549+00:00 → 2025-12-16 00:13:41.468549+00:00

Clean dataset saved to → C:\Users\HP ELITEBOOK 830 G5\Desktop\cybox-pay\data\cleaned_transactions_for_eda.csv
